# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\L22 fiducial')
mouse_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\Mouse 1596')

registered_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\Registered')

In [3]:
microscopy_path = Path(mouse_dir, 'SHG Mouse 1596 _ down8x.tif')
microscopy_tile_path = Path(mouse_dir, 'SHG Mouse 1596 settings.ome.tif')

us_path = Path(mouse_dir, 'L22 Mouse 1596 Z16727.tif')
us_pl_path = Path(mouse_dir, 'L22 Mouse 1596 Z16727.pos')
us_param_path = Path(mouse_dir, 'L22 Mouse 1596 Z16727_Run-1_Settings.mat')

transform_path = Path(fiducial_dir, 'L22 Transform.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [10]:
spacing = [18.48, 18.48, 18.48]
dynamic_range = 80
gauge_height = 16727
us_image = coord.open_us(us_path, us_pl_path, us_param_path, spacing, gauge_height, dynamic_range)

In [5]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

Load the coordinate transform

In [19]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)
    
coordinate_transform[3] = coordinate_transform[3] - 4000

Apply the registration and save the results

In [20]:
us_reg_path = Path(registered_dir, 'Mouse 1596 - L22 to SHG.tif')
mpm_reg_path = Path(registered_dir, 'Mouse 1596 - SHG to L22.tif')

In [ ]:
us_reg = tran.apply_transform_params(microscopy_image, us_image, 
                                     coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [21]:
mpm_reg = tran.apply_transform_params(us_image, microscopy_image, 
                                     -1*coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [ ]:
sitk.WriteImage(us_reg, str(us_reg_path))

In [24]:
sitk.WriteImage(sitk.Cast(mpm_reg, sitk.sitkUInt8), str(mpm_reg_path))

In [25]:
us_rotated_path = Path(registered_dir, 'Mouse 1596 - L22 rotated.tif')
sitk.WriteImage(sitk.Cast(us_image, sitk.sitkUInt8), str(us_rotated_path))